# IB API_Connect-get-data-Save-to-disk

**This script will download OHLCV historical data from the Interactive Brokers IB TWS API. Input a list of symbols stored in a CSV file.  Can be used for any security type and frequency offered by IB.  Downloaded data is stored to individual CSV files. 2-16-19, 10-14-19. Script works with TWS API -9.72 and higher.  IB TWS software must be installed and running with IB account.  Script is intended for all instrument types however to date it has been tested only on 'CASH'**<BR>
Be sure to install:   pip3 install -U ib_insync<BR>
<BR>


&copy; Matt Chmielewski (emskiphoto) | mattcbike@gmail.com<BR>
https://github.com/emskiphoto/IB-API

References:<BR>
https://github.com/erdewit/ib_insync/blob/master/notebooks/basics.ipynb

https://interactivebrokers.github.io/tws-api/connection.html

https://groups.io/g/insync

In [1]:
import ib_insync
import numpy as np
import pandas as pd
import os
import time
from pathlib import Path
# print(ib_insync.__all__)

In [2]:
from ib_insync import *
util.startLoop()

In [3]:
#                   MUST LOG IN TO 'TWS' DESKTOP PROGRAM BEFORE RUNNING THIS CELL (paper account)                  ##

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=1)
# Default values:  ib.connect('127.0.0.1', 7497, clientId=1)
# Succesful connection will return:  "<IB connected to 127.0.0.1:7497 clientId=XX>"
# Live Trading ports: TWS: 7496; IB Gateway: 4001. 
# Simulated Trading ports: TWS: 7497; IB Gateway: 4002"

<IB connected to 127.0.0.1:7497 clientId=1>

Save a list of instruments to be downloaded as a CSV file, one column only, with header "Symbol".  In Windows explorer navigate to path of CSV file. Use ALT + D, CTRL + C to copy filepath.   Paste filepath as answer to next question.  <BR>
    <BR>
Filepath must terminate with "\ticker_lists\instrument_type", for example:<BR>
        C:\Users\user\OneDrive\ticker_lists\CASH<BR>
        <BR>
instrument_list = ("STK", "OPT", "FUT", "IND", "FOP", "CASH", "BAG", "WAR",
                   "BOND", "CMDTY", "NEWS", "FUND")          

In [38]:
# Request filepath to ticker list copied directly from Windows Explorer
# C:\Users\user\OneDrive\STOCKS\TICKER_HISTORIC_FILES\Python_Data\ticker_lists\CASH
pasted_path = input('Paste Windows file path to ticker list: \n')
source_path = Path(pasted_path)   #convert Windows path format to Python format
os.chdir(source_path)             #Set source_path as current working directory
source_path

Paste Windows file path to ticker list: 
C:\Users\user\OneDrive\STOCKS\TICKER_HISTORIC_FILES\Python_Data\ticker_lists\CASH


WindowsPath('C:/Users/user/OneDrive/STOCKS/TICKER_HISTORIC_FILES/Python_Data/ticker_lists/CASH')

In [28]:
# Define financial instrument type by parsing filepath

instrument_list = ("STK", "OPT", "FUT", "IND", "FOP", "CASH", "BAG", "WAR",
                   "BOND", "CMDTY", "NEWS", "FUND")
# Option, Future, Forex, Index, CFD, Commodity,Bond, FuturesOption, MutualFund or Warrant, etc.
security_type = ''

for i in instrument_list:
    if i in str(source_path):
        security_type = i
        print(i)
        break
    else:
        pass

security_type

CASH


'CASH'

Duration (range) of time of data to be covered in data:<BR>
    
    Unit	Description
    S	Seconds
    D	Day
    W	Week
    M	Month
    Y	Year
 Example:   "10 D" is ten days

In [27]:
# Define data frequency using IB 'barSizeSetting' format.  This will also define the filepath
# where exported data will be stored (Ex. 1 secs, 5 secs, 10 secs, 15 secs, 30 secs,
# 1 min, 2 mins, 3 mins, 5 mins, 10 mins, 15 mins, 20 mins, 30 mins, 1 hour, 2 hours, 3 hours, 4 hours, 8 hours,
# 1 day, 1 week, 1 month)
data_frequency = '1 hour'
data_duration = '6 M'

In [44]:
# define locations and paths
security_type_path = security_type + "/"
ticker_lists_root = "C:/Users/user/OneDrive/STOCKS/TICKER_HISTORIC_FILES/Python_Data/ticker_lists/"
ticker_list_name_partial ="ticker_list"
# ticker_list_name = ticker_list_name_partial + ".csv"
ticker_list_name = pasted_path + '/' + ticker_list_name_partial + ".csv"
data_output_root = "C:/Users/user/OneDrive/STOCKS/TICKER_HISTORIC_FILES/Python_Data/"

In [50]:
#define list of tickers to pull based on list in CSV, column header of list must be 'Symbol'
ticker_list_csv = pd.read_csv(ticker_list_name)
ticker_list = ticker_list_csv.Symbol.tolist() 

In [51]:
# read from list of tickers to be download
ticker_list[2]

'AUDCNH'

https://interactivebrokers.github.io/tws-api/historical_bars.html

In [52]:
#*****OPTIONAL****** request historical data for a single ticker in ticker_list *****OPTIONAL*****
contract = Forex(ticker_list[2])
bars = ib.reqHistoricalData(contract, endDateTime='', durationStr='10 D',
        barSizeSetting=data_frequency, whatToShow='MIDPOINT', useRTH=True)
contract

print(bars[:3])
print("\n")
print(bars[-3])

[BarData(date=datetime.datetime(2019, 10, 1, 16, 15), open=4.792705, high=4.792705, low=4.78852, close=4.790985, volume=-1, average=-1.0, barCount=-1), BarData(date=datetime.datetime(2019, 10, 1, 17, 0), open=4.790985, high=4.793415, low=4.789765, close=4.79099, volume=-1, average=-1.0, barCount=-1), BarData(date=datetime.datetime(2019, 10, 1, 18, 0), open=4.79099, high=4.797515, low=4.79049, close=4.795285, volume=-1, average=-1.0, barCount=-1)]


BarData(date=datetime.datetime(2019, 10, 14, 20, 0), open=4.787385, high=4.80053, low=4.786045, close=4.797685, volume=-1, average=-1.0, barCount=-1)


In [59]:
# how many tickers are in ticker list?
ticker_quantity = len(ticker_list)

In [65]:
# request historical data for tickers in ticker_list starting with 'start_ticker_row' and ending at 'end_ticker_row'
# using start and end ticker values makes it possible to limit downloads to only some of tickers on ticker_list

start_ticker_row = 61
end_ticker_row = 66
total_tickers = end_ticker_row - start_ticker_row

contracts = [Forex(pair) for pair in ticker_list[start_ticker_row:end_ticker_row]]
ib.qualifyContracts(*contracts)

[Forex('GBPSEK', conId=28027122, exchange='IDEALPRO', localSymbol='GBP.SEK', tradingClass='GBP.SEK'),
 Forex('GBPSGD', conId=290880559, exchange='IDEALPRO', localSymbol='GBP.SGD', tradingClass='GBP.SGD'),
 Forex('GBPTRY', conId=41972332, exchange='IDEALPRO', localSymbol='GBP.TRY', tradingClass='GBP.TRY'),
 Forex('GBPUSD', conId=12087797, exchange='IDEALPRO', localSymbol='GBP.USD', tradingClass='GBP.USD'),
 Forex('GBPZAR', conId=208558332, exchange='IDEALPRO', localSymbol='GBP.ZAR', tradingClass='GBP.ZAR')]

In [66]:
ib.qualifyContracts(*contracts)[2]

Forex('GBPTRY', conId=41972332, exchange='IDEALPRO', localSymbol='GBP.TRY', tradingClass='GBP.TRY')

In [67]:
print(contracts[1].symbol + contracts[1].currency)   #only works for Forex (CASH)
print('{}.csv'.format(contract.currency + contract.symbol))   #only works for Forex (CASH)

GBPSGD
CNHAUD.csv


In [73]:
# define directory to be created based on current date.   
# Historical data CSV files will be stored here.  
# Name format is "/data frequency/security type/YearMonthDayHourMinutes/"
datestring = time.strftime("%Y%m%d%H%M")
# data_output_filepath = data_output_root + data_frequency + "/" + security_type + "/" + datestring + "/"
print(str(len(contracts)) + ' contracts will be saved to: \n' + data_output_filepath)

5 contracts will be saved to: 
C:/Users/user/OneDrive/STOCKS/TICKER_HISTORIC_FILES/Python_Data/1 hour/CASH/201910142218/


In [74]:
# Create folders and filenames to store CSVs based on current date and time.   

datestring = time.strftime("%Y%m%d%H%M")   #"%Y%m%d%H%M%S"
data_output_filepath = data_output_root + data_frequency + "/" + security_type + "/" + datestring + "/"

# if contract doesn't exist already, make output directory
# if not os.path.exists(data_output_filepath + '{}.csv'.format(contract.currency + contract.symbol)):
if not os.path.exists(data_output_filepath):
#         os.makedirs(data_output_filepath)
    os.mkdir(data_output_filepath)
print(str(len(contracts)) + ' contracts will be saved to: \n' + data_output_filepath)    

5 contracts will be saved to: 
C:/Users/user/OneDrive/STOCKS/TICKER_HISTORIC_FILES/Python_Data/1 hour/CASH/201910142219/


####FUTURE IMPROVEMENT 8/24/19: <BR>
    improve I/O speed by implementing HDFStore tool <BR>
    http://localhost:8888/notebooks/py4fi/code/ch09/09_input_output.ipynb
    h5s = pd.HDFStore(filename + '.h5s', 'w')

####FUTURE IMPROVEMENT 8/24/19: <BR>
add more Historical Data Types, like 'HISTORICAL_VOLATILITY', 'TRADES' : <BR>
    https://interactivebrokers.github.io/tws-api/historical_bars.html#hd_what_to_show

In [75]:
# request data for all tickers in tickerlist and save to individual .CSV files.   This will overwrite existing files.
# remove columns, format significant digits of data

OHLC = ['open','high','low','close']
columns_to_drop = ['barCount', 'average']
for contract in contracts: 
        # Send request to Interactive Brokers TWS for contracts
        bars = ib.reqHistoricalData(contract, endDateTime='', durationStr=data_duration,
            barSizeSetting=data_frequency, whatToShow='MIDPOINT', useRTH=True)
        
        # Create dataframe from 'bars' list output, format bars data using IB "util"
        df = util.df(bars) 
        df.set_index('date')
        df[OHLC] = df[OHLC].astype('float').round(3)  #round values to 3 significant digits
        if security_type == 'CASH':
#             df.drop([columns_to_drop], axis=1, inplace=True, errors='ignore')  #Remove useless columns
            df.drop(['volume', 'barCount', 'average'], axis=1, inplace=True)  #Remove useless columns
        source_filename = contract.currency + contract.symbol
        df.to_csv(data_output_filepath + '{}.csv'.format(source_filename), index=False)

Disconnecting
The following will disconnect ib and clear all its state:

In [76]:
ib.disconnect()

### END *******************END************END********END***********<BR>
    This is the END END END END of this script